In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from PyQt5 import QtWidgets, QtCore
import PyQt5 as pg
import scipy
from scipy.fftpack import fft
import time
from tkinter import TclError
from scipy.io import wavfile as wav
import wave
import soundfile as sf
from pydub import AudioSegment
from pyqtgraph.Qt import QtWidgets, QtCore
import math
from collections import Counter
from pydub.utils import get_array_type
import array
from pydub.silence import split_on_silence


/usr/local/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
   # White keys are in Uppercase and black keys (sharps) are in lowercase
octave = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] 
base_freq = 440 #Frequency of Note A4
keys = np.array([x+str(y) for y in range(0,9) for x in octave])
# Trim to correct # keys
start = np.where(keys == 'A0')[0][0]
end = np.where(keys == 'C8')[0][0]
keys = keys[start:end+1]

note_freqs = list(([2**((n+1-49)/12)*base_freq for n in range(len(keys))]))

In [3]:
c_major_scale_freqs=[]
c_major_scale=['C4', 'D4', 'E4', 'F4', 'G4']
for i in range(len(keys)):
    for j in range(5):
        if c_major_scale[j]==keys[i]:
            c_major_scale_freqs.append(note_freqs[i])
print(c_major_scale_freqs)

[261.6255653005986, 293.6647679174076, 329.6275569128699, 349.2282314330039, 391.99543598174927]


In [4]:
def closest_value(input_list, input_value):

  arr = np.asarray(input_list)

  i = (np.abs(arr - input_value)).argmin()

  return arr[i]

In [5]:
#%matplotlib tk

# constants
CHUNK = 1024*2            # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second
RECORD_SECONDS = 3


In [6]:
os.remove('/Users/emmarobinson/Downloads/piano_audio.wav') #change this according to your computer file
os.remove('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file
# create matplotlib figure and axes
#fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance

p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input_device_index =3, #change this according to the mic port on your computer
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)
frames = []
print("* recording")
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    stream.write(data, CHUNK)
    frames.append(data)

print("* done")

stream.stop_stream()
stream.close()

p.terminate()
file_path='/Users/emmarobinson/Downloads/piano_audio.wav' #change this according to your computer file
file_name = file_path.split('/')[-1]
wf = wave.open(file_path, "wb")
# set the channels
wf.setnchannels(CHANNELS)
# set the sample format
wf.setsampwidth(p.get_sample_size(FORMAT))
# set the sample rate
wf.setframerate(RATE)
# write the frames as bytes
wf.writeframes(b"".join(frames))
# close the file
wf.close()


* recording
* done


In [7]:
sound = AudioSegment.from_file(file_path, format = 'wav') 
audio_chunks = split_on_silence(sound
                            ,min_silence_len = 100
                            ,silence_thresh = -45
                            ,keep_silence = 50
                        )

# Putting the file back together
combined = AudioSegment.empty()
for chunk in audio_chunks:
    combined += chunk
combined.export(f'/Users/emmarobinson/Downloads/piano_audio_new.wav', format = 'wav') #change this according to your computer file

<_io.BufferedRandom name='/Users/emmarobinson/Downloads/piano_audio_new.wav'>

In [8]:
rate, data = wav.read('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file
audio_segment = AudioSegment.from_file('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file

duration = len(audio_segment)/1000

# calculate the length of our chunk in the np.array using sample rate
chunk = int(rate * duration)

# length of delimiting 1600hz tone
offset = int(rate * 0.005)

# number of bits in the audio data to decode
bits = int(len(data) / chunk)

def get_freq(bit):
    # start position of the current bit
    strt = (chunk * bit) 
    
    # remove the delimiting 1600hz tone
    end = (strt + chunk) 
   
    
    # slice the array for each bit
    sliced = data[strt:end]

    w = np.fft.fft(sliced)
    w=np.absolute(w)
    freqs = np.fft.fftfreq(len(w))

    

    # Find the peak in the coefficients
    idx = np.argmax(np.abs(w))
    freq = freqs[idx]
    freq_in_hertz = abs(freq * rate)
    return freq_in_hertz

decoded_freqs = [get_freq(bit) for bit in range(bits)]

In [9]:
for i in range(len(decoded_freqs)):
    note=closest_value(note_freqs,decoded_freqs[i])
    index = note_freqs.index(note)
    print(keys[index])

C4


In [10]:
for i in range(len(decoded_freqs)):
    note=closest_value(note_freqs,decoded_freqs[i])
    index = note_freqs.index(note)
    played_note=keys[index]

x=0
if played_note==c_major_scale[x]:
    print("yay")
else:
        print("try again")

yay


In [31]:
'''
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK,exception_on_overflow = False)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = np.frombuffer(data, dtype='h')  
    data_np = np.array(data_int, dtype='h')/140 + 255
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
                # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break
    '''

NameError: name 'ax1' is not defined

In [5]:
audio_segment = AudioSegment.from_file('/Users/emmarobinson/Downloads/c-major-scale.wav')
# Print attributes
print(f"Channels: {audio_segment.channels}")
print(f"Sample width: {audio_segment.sample_width}")
print(f"Frame rate (sample rate): {audio_segment.frame_rate}")
print(f"Frame width: {audio_segment.frame_width}")
print(f"Length (ms): {len(audio_segment)}")
print(f"Frame count: {audio_segment.frame_count()}")
print(f"Intensity: {audio_segment.dBFS}")

Channels: 1
Sample width: 2
Frame rate (sample rate): 8000
Frame width: 2
Length (ms): 8001
Frame count: 64004.0
Intensity: -3.0108529080336788


In [33]:
for i in range(len(decoded_freqs)):
    note=closest_value(note_freqs,decoded_freqs[i])
    index = note_freqs.index(note)
    print(keys[index])



C8
